In [1]:
from datasets import load_dataset

/nlp/scr/jiangm/miniconda3/envs/plaid/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from large_datasets import PileDataset
from sampler import Sampler

dataset = PileDataset("test_data")
sampler = Sampler(dataset)

In [2]:
sampler.random_sample(5)

Using 32 workers for sampling.


['{"text": "South Arabia during World War I\\n\\nThe campaign in South Arabia during World War I was a minor struggle for control of the port city of Aden, an important way station for ships on their way from Asia to the Suez Canal. The British Empire declared war on the Ottoman Empire on 5 November 1914, and the Ottomans responded with their own declaration on 11 November. From the beginning, the Ottomans had planned an invasion of Britain\'s Aden Protectorate in cooperation with the local Arab tribes. The Ottomans had gathered in some strength on the Cheikh Sa\\u00efd, a peninsula which juts out into the Red Sea towards the island of Perim.\\n\\nAt the start of the war, the British had one force stationed in the Aden Protectorate, the Aden Brigade, which was part of the British Indian Army. In November 1914, an Ottoman force from Yemen attacked Aden, but was driven off by the Brigade.\\n\\nLand campaign\\n\\nCapture of Cheikh Sa\\u00efd\\nThe 29th Indian Brigade, under Brigadier-Gene

In [3]:
sampler.random_sample(5, category="Pile-CC")

Using 32 workers for sampling.


['{"text": "Graduate Engineer Salaries at Schneider Electric\\n\\na Graduate Engineer at Schneider Electric earns a salary between \\u20b9 5,51,000 to \\u20b9 5,93,000 per annum, with an average of \\u20b9 5,72,000. Ranging from \\u20b9 5,61,500 at the 25th percentile to \\u20b9 5,82,500 at the 75th percentile, with top earners earning more than \\u20b9 5,88,800", "meta": {"pile_set_name": "Pile-CC"}}',
 '{"text": "Solar electricity isn\'t the only renewable energy whipping boy out there. Wind power has also taken more than its share of lumps, frequently saddled with a reputation for excessive noise and energy inefficiency. Plus, if some of the rumors are true, wind harvesters of the world have steadily been turning the planet\'s bird population into an airborne puree of blood and feathers.\\n\\nTo be fair, wind turbines do kill birds -- but so do vehicles, skyscrapers, pollution and the introduction of invasive species into their habitats. Humans have had bird blood on their hands for

In [1]:
from refactor import LargeDataset, Sampler

dataset = LargeDataset("test_data")
sampler = Sampler(dataset)

Processing: 100%|██████████| 55.6M/55.6M [00:00<00:00, 162MB/s]
Processing: 100%|██████████| 56.0M/56.0M [00:00<00:00, 153MB/s]


In [3]:
sampler.random_sample(5)

Using 32 workers for sampling.


['{"text": "South Arabia during World War I\\n\\nThe campaign in South Arabia during World War I was a minor struggle for control of the port city of Aden, an important way station for ships on their way from Asia to the Suez Canal. The British Empire declared war on the Ottoman Empire on 5 November 1914, and the Ottomans responded with their own declaration on 11 November. From the beginning, the Ottomans had planned an invasion of Britain\'s Aden Protectorate in cooperation with the local Arab tribes. The Ottomans had gathered in some strength on the Cheikh Sa\\u00efd, a peninsula which juts out into the Red Sea towards the island of Perim.\\n\\nAt the start of the war, the British had one force stationed in the Aden Protectorate, the Aden Brigade, which was part of the British Indian Army. In November 1914, an Ottoman force from Yemen attacked Aden, but was driven off by the Brigade.\\n\\nLand campaign\\n\\nCapture of Cheikh Sa\\u00efd\\nThe 29th Indian Brigade, under Brigadier-Gene

In [4]:
sampler.random_sample(5, category="Pile-CC")

Using 32 workers for sampling.


['{"text": "Graduate Engineer Salaries at Schneider Electric\\n\\na Graduate Engineer at Schneider Electric earns a salary between \\u20b9 5,51,000 to \\u20b9 5,93,000 per annum, with an average of \\u20b9 5,72,000. Ranging from \\u20b9 5,61,500 at the 25th percentile to \\u20b9 5,82,500 at the 75th percentile, with top earners earning more than \\u20b9 5,88,800", "meta": {"pile_set_name": "Pile-CC"}}',
 '{"text": "Solar electricity isn\'t the only renewable energy whipping boy out there. Wind power has also taken more than its share of lumps, frequently saddled with a reputation for excessive noise and energy inefficiency. Plus, if some of the rumors are true, wind harvesters of the world have steadily been turning the planet\'s bird population into an airborne puree of blood and feathers.\\n\\nTo be fair, wind turbines do kill birds -- but so do vehicles, skyscrapers, pollution and the introduction of invasive species into their habitats. Humans have had bird blood on their hands for

In [3]:
train_dataset = load_dataset("monology/pile-uncopyrighted", cache_dir="data/pile", streaming=True, split="train")
val_dataset = load_dataset("monology/pile-uncopyrighted", cache_dir="data/pile", streaming=True, split="validation")

In [7]:
train_length = sum(1 for _ in train_dataset)

In [8]:
train_length

177009652

In [5]:
val_length = sum(1 for _ in val_dataset)	


In [6]:
val_length

179996